In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    #check if image is grayscale or color
    shape = img.shape
    if(len(shape) == 3):               #its a color image
        mask_color = (255,)*shape[-1]   #shape[-1] = no. at last index
    else:                              #otherwise its a gray image
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [3]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def filter_color(img):
    yellow_min = np.array([50, 80, 80], np.uint8)
    yellow_max = np.array([60, 255, 255], np.uint8)
    yellow_mask = cv2.inRange(img, yellow_min, yellow_max)

    white_min = np.array([200, 0, 200], np.uint8)
    white_max = np.array([255, 0, 255], np.uint8)
    white_mask = cv2.inRange(img, white_min, white_max)
    
    yelWhtMerge = cv2.bitwise_or(yellow_mask, white_mask)
    img = cv2.bitwise_and(img, img, yelWhtMerge)
    return img

In [5]:
cap = cv2.VideoCapture('../dataset/day-lane.mp4')

#get height and width of frame
ret, frame = cap.read()
height, width, channel = frame.shape

#keeping the portion of image which contains lane
topLeftPt = (width*0.30, height*0.60)
topRightPt = (width*0.70, height*0.60)

#polygon has 6 vertices
region_of_interest_points = [
(0, height*0.90),
#(0, height*0.60),
topLeftPt,
topRightPt,
#(width, height*0.60),
(width, height*0.90),
]

i = 1  #image count
while(cap.isOpened()):
    # Take each frame
    ret, frame = cap.read()
    '''
    #increase intensity by I = R + G -B
    frame = frame[:,:,2] + frame[:,:,1] - frame[:,:,0] 
    canny = cv2.Canny(frame, 171, 255)
    cv2.imshow('intense', frame)
    
    #crop image
    cropped = region_of_interest(canny, np.array([region_of_interest_points], np.int32))
    cv2.imshow('cropped', cropped)
    '''
    cv2.imshow('denoise', frame)
    k = cv2.waitKey(25) & 0xFF
    if k == ord('q'):
        break
    elif k == ord('s'):
        cv2.imwrite('dataset/ld'+str(i)+'.jpg', frame)
        i += 1

cap.release()
cv2.destroyAllWindows()

# Custom color

In [28]:
img = np.zeros((15,15,3), dtype = np.uint8)

img[:,:,1] = 255   #saturation
img[:,:,2] = 255   #value
j = 1
#changing hue
for i in range(0, 256, 100):
    img[:,:,0] = i
    #cvt to rgb
    color = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imshow('y', color)
    j += 1
    